<a href="https://colab.research.google.com/github/Ashutosh9826/Machine_Learning/blob/main/Concrete_Strength_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multiple Linear Regression

## Importing the libraries

In [130]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Importing the dataset

In [131]:
dataset1 = pd.read_csv('train.csv')
dataset2 = pd.read_csv('test.csv')

def normalize_binary_column(col):
    return col.astype(str).str.strip().str.lower().map({
        'yes': 1, 'y': 1, '1': 1,
        'no': 0, 'n': 0, '0': 0 ,
    }).fillna(0).astype(int)

dataset1['is_approved'] = normalize_binary_column(dataset1['is_approved'])
dataset2['is_approved'] = normalize_binary_column(dataset2['is_approved'])

dataset1['cement_grade'] = dataset1['cement_grade'].replace('fifty three', '53')
dataset2['cement_grade'] = dataset2['cement_grade'].replace('fifty three', '53')

dataset1['is_valid_strength'] = dataset1['is_valid_strength'].replace({True: 1, False:0})
dataset2['is_valid_strength'] = dataset2['is_valid_strength'].replace({True: 1, False:0})

X_train = dataset1.drop(dataset1.columns[[8,13,14,19,20,21,23,24,28,29,31,32,35,36]], axis=1)
y_train = dataset1.iloc[:, 8]
X_test = dataset2.drop(dataset2.columns[[8,13,14,19,20,21,23,24,28,29,31,32,35,36]], axis=1)
y_test = dataset2.iloc[:, 8]

X = pd.concat([X_train, X_test], axis=0, ignore_index=True)
y = pd.concat([y_train, y_test], axis=0, ignore_index=True)

/tmp/ipython-input-188532627.py:16: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset1['is_valid_strength'] = dataset1['is_valid_strength'].replace({True: 1, False:0})
/tmp/ipython-input-188532627.py:17: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset2['is_valid_strength'] = dataset2['is_valid_strength'].replace({True: 1, False:0})


In [132]:
# print(X)

In [133]:
# print(y)

#Encoding

In [134]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(sparse_output=False), [12,13,14,17,20])], remainder='passthrough')
X = (ct.fit_transform(X))

In [135]:
# print(X)

In [136]:
print(np.isinf(X).sum())
X[np.isinf(X)] = np.nan
print(np.isinf(X).sum())

18
0


#Missing Values

In [137]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer.fit(X[:, 0:-1])
X[:, 0:-1] = imputer.transform(X[:, 0:-1])

In [138]:
imputer2 = SimpleImputer(missing_values = np.nan, strategy = 'mean')
y = y.values.reshape(-1, 1)
imputer2 = SimpleImputer(missing_values=np.nan, strategy='mean')
y = imputer2.fit_transform(y)

#spliting the dataset

In [139]:
print(X.shape)
print(y.shape)

(974, 2780)
(974, 1)


In [140]:
X_train = X[:865]
X_test = X[865:]
y_train = y[:865]
y_test = y[865:]

In [141]:
# print(X_train)

# Adding New Feature "binder_per_leak_strength"

In [142]:
binder_per_leak_strength = X_train[:, 2771] / X_train[:, 2774]
X_train = np.hstack((X_train, binder_per_leak_strength.reshape(-1, 1)))
# print(X_train)
binder_per_leak_strength = X_test[:, 2771] / X_test[:, 2774]
X_test = np.hstack((X_test, binder_per_leak_strength.reshape(-1, 1)))
# print(X_test)

## Training the Multiple Linear Regression model on the Training set

In [143]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression()

## Predicting the Test set results

In [144]:
y_pred = regressor.predict(X_test)
np.set_printoptions(precision=2)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[1.38 1.31]
 [1.53 1.63]
 [1.2  1.2 ]
 [1.35 1.35]
 [1.64 1.56]
 [1.   1.08]
 [1.77 1.96]
 [1.15 1.11]
 [1.22 1.19]
 [1.17 1.17]
 [7.06 6.76]
 [1.51 1.38]
 [2.01 2.17]
 [1.45 1.45]
 [1.8  1.8 ]
 [1.37 1.3 ]
 [1.43 1.4 ]
 [1.11 1.23]
 [1.09 1.14]
 [1.19 1.21]
 [1.21 1.31]
 [1.16 1.2 ]
 [1.38 1.39]
 [1.83 1.8 ]
 [1.43 1.39]
 [1.46 1.47]
 [1.96 1.84]
 [1.52 1.61]
 [1.46 1.6 ]
 [1.27 1.27]
 [1.74 1.83]
 [1.26 1.4 ]
 [1.27 1.35]
 [1.16 1.17]
 [1.62 1.61]
 [1.83 1.8 ]
 [1.74 1.69]
 [1.16 1.26]
 [1.7  1.71]
 [1.65 1.56]
 [8.68 8.59]
 [1.15 1.26]
 [0.98 1.06]
 [1.07 1.1 ]
 [1.63 1.6 ]
 [1.27 1.37]
 [1.42 1.38]
 [1.9  1.76]
 [7.94 8.4 ]
 [1.13 1.16]
 [1.91 1.86]
 [1.64 1.66]
 [1.35 1.27]
 [1.45 1.45]
 [1.56 1.6 ]
 [1.57 1.71]
 [1.41 1.34]
 [1.25 1.24]
 [1.45 1.4 ]
 [1.63 1.6 ]
 [1.36 1.3 ]
 [1.89 1.73]
 [1.52 1.44]
 [1.25 1.39]
 [1.01 1.13]
 [1.22 1.32]
 [1.81 1.83]
 [1.27 1.2 ]
 [1.83 1.9 ]
 [1.18 1.3 ]
 [1.52 1.44]
 [1.13 1.16]
 [1.36 1.26]
 [1.61 1.6 ]
 [1.58 1.48]
 [1.78 1.9 ]
 [1.27 1.27]

## Evaluating the Model Performance

In [145]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
r2_score(y_test, y_pred)

0.9934800695690761